## Conversion to ONNX from keras model using tf2onnx python api

In [1]:
# get image
!wget -q https://raw.githubusercontent.com/onnx/tensorflow-onnx/main/tests/ade20k.jpg

In [3]:
!pip install tensorflow tf2onnx onnxruntime -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00


In [4]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import onnxruntime

img_path = 'ade20k.jpg'

img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

### Run the keras model

In [8]:
model = ResNet50(weights='imagenet')

preds = model.predict(x)
print('Keras Predicted:', decode_predictions(preds, top=3)[0])
model.save(os.path.join("./tmp", model.name))

1/1 [==============================] - 1s 1s/step


Keras Predicted: [('n04285008', 'sports_car', 0.34477925), ('n02974003', 'car_wheel', 0.2876423), ('n03100240', 'convertible', 0.10070901)]


### Convert to ONNX using the Python API

In [9]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = model.name + ".onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

### Run the ONNX model

In [10]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": x})

print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])

# make sure ONNX and keras have the same results
np.testing.assert_allclose(preds, onnx_pred[0], rtol=1e-5)

ONNX Predicted: [('n04285008', 'sports_car', 0.34477764), ('n02974003', 'car_wheel', 0.2876437), ('n03100240', 'convertible', 0.100708835)]


### Convert to ONNX using the command line

In [13]:
!python -m tf2onnx.convert --opset 13 \
    --saved-model {os.path.join("./tmp", model.name)} \
    --output  {os.path.join("./output", model.name + ".onnx")}

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-05-31 16:20:59,807 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-05-31 16:21:09,036 - INFO - Signatures found in model: [serving_default].
2024-05-31 16:21:09,036 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-05-31 16:21:09,038 - INFO - Output names: ['predictions']
2024-05-31 16:21:17,488 - INFO - Using tensorflow=2.15.0, onnx=1.16.1, tf2onnx=1.16.1/15c810
2024-05-31 16:21:17,488 - INFO - Using opset <onnx, 13>
2024-05-31 16:21:18,064 - INFO - Computed 0 values for constant folding
2024-05-31 16:21:19,240 - INFO - Optimizing ONNX model
2024-05-31 16:21:21,542 - INFO - After optimization: Add -1 (18->17), BatchNormalization -53 (53->0), Const -161 (271->110), GlobalAveragePoo